In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

In [2]:
tdf = pd.read_csv('../data/ks-projects-201801.csv')

In [3]:
df_1_9_kickstarter = [pd.read_csv(f'../data/kickstarter_2018/Kickstarter00{x}.csv') for x in range(1,10)]

df_1 = pd.concat(df_1_9_kickstarter)


In [4]:
df_1.reset_index(inplace = True)

In [5]:
df_10_53_kickstarter = [pd.read_csv(f'../data/kickstarter_2018/Kickstarter0{x}.csv') for x in range(10,57)]

df_10 = pd.concat(df_10_53_kickstarter)


In [6]:
df_10.reset_index(inplace = True)

In [7]:
list_df = [df_1,df_10]

In [8]:
kdf = pd.concat(list_df)

In [9]:
kdf.drop(columns = 'index',inplace = True)

## Getting all IDs not in tdf to copy the columns and then merge to one df.

In [10]:
tdf_notin = tdf[(tdf['ID'].isin(kdf['id'])).map({False:True,True:False})].copy()

In [11]:
tdf_notin['spotlight'] = 'unknown'
tdf_notin['staff_pick'] = 'unknown'

In [12]:
kdf['name'] = kdf['name'] + ' ' + kdf['blurb']

In [13]:
kdf['main_category'] = kdf['category'].map(lambda x : x.split(':')[3].split(',')[0][1:-1].split('/')[0])

In [14]:
kdf['category'] = kdf['category'].map(lambda x : x.split(':')[3].split(',')[0][1:-1].split('/')[-1])

In [15]:
kdf['launched']=kdf['launched_at'].map(lambda x : datetime.fromtimestamp(x))
kdf['deadline']=kdf['deadline'].map(lambda x : datetime.fromtimestamp(x))

In [16]:
tdf_notin['launched'] = tdf_notin['launched'].map(lambda x : pd.to_datetime(x))
tdf_notin['deadline'] = tdf_notin['deadline'].map(lambda x : pd.to_datetime(x))

In [17]:
kdf['backers'] = kdf['backers_count']
tdf_notin['id'] = tdf_notin['ID']

In [18]:
kdf['pledged'] = kdf['pledged'].map(lambda x : round(x,2))
kdf['usd_pledged'] = kdf['usd_pledged'].map(lambda x : round(x,2))

In [19]:
tdf_notin['usd_pledged'] = tdf_notin['usd pledged']

In [20]:
kdf.drop_duplicates(['id'],inplace = True)

In [21]:
kdf = kdf[['creator','id','name','category','main_category','currency','launched','deadline','pledged','usd_pledged','goal','backers','country','spotlight','staff_pick','state']].copy()

In [22]:
kdf.to_csv('./unique_ks.csv')

In [23]:
tdf_notin = tdf_notin[['id', 'name', 'category', 'main_category', 'currency','launched','deadline','pledged','usd_pledged',
                       'goal','backers','country','spotlight','staff_pick','state']].copy()

In [24]:
kdf = kdf[['id','name','category','main_category','currency','launched','deadline','pledged','usd_pledged',
           'goal','backers','country','spotlight','staff_pick','state']].copy()

In [25]:
df = pd.concat([kdf,tdf_notin])

# Total Dataframe

In [26]:
df = df.loc[(df['state'] == 'failed') | (df['state'] == 'successful')].copy()

In [27]:
df['main_category'] = df['main_category'].map(lambda x : x.lower())

In [28]:
df['category'] = df['category'].map(lambda x : x.lower())

In [29]:
df.reset_index(drop = True,inplace = True)

In [30]:
df['spotlight'] = df['spotlight'].map({True : 'yes', False : 'no','unknown':'unknown'})
df['staff_pick'] = df['staff_pick'].map({True : 'yes', False : 'no','unknown':'unknown'})

In [31]:
df.to_csv('../data/total_kickstarter.csv',index = False)

## Reading in new data

In [32]:
df = pd.read_csv('../data/total_kickstarter.csv')

In [33]:
df['result'] = df['state'].map({'successful':1,'failed':0})
df.drop(columns = 'state',inplace = True)

In [34]:
df['launched'] = pd.to_datetime(df['launched'])
df['deadline'] = pd.to_datetime(df['deadline'])

Feature Engineering technically

In [35]:
df['duration'] = df['deadline'] - df['launched']

In [36]:
df['name'] = df['name'].map(lambda x : re.sub("[^a-zA-Z]", " ",str(x)).lower())

In [37]:
df['duration'] = df['duration'].map(lambda x : x.days)

In [38]:
df['month_launched'] = df['launched'].map(lambda x : x.month)

In [39]:
df['month_launched'] = df['month_launched'].map({
    1:'jan',
    2:'feb',
    3:'mar',
    4:'apr',
    5:'may',
    6:'jun',
    7:'july',
    8:'aug',
    9:'sep',
    10:'oct',
    11:'nov',
    12:'dec',
})

In [40]:
df.to_csv('../data/total_kickstarter.csv',index = False)

In [41]:
model_df = pd.get_dummies(data = df,columns = ['currency','main_category','category','country',
                                         'spotlight','staff_pick','month_launched','duration'])

In [43]:
model_df.to_csv('../data/no_cvec_kickstarter.csv',index = False)

In [ ]:
# df = pd.read_csv('./data/no_cvec_kickstarter.csv')